In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
from tqdm.auto import tqdm

In [2]:
with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

 ### Starting a single-node Docker container (for basic local testing)

- You can directly run a single Elasticsearch container without Docker Compose.
- Ensure Docker is installed and allocated enough memory in Codespaces' Docker Desktop settings (at least 4GB is recommended).
- Open a terminal in your Codespace.
- Create a Docker network: docker network create elastic.
- Pull the Elasticsearch Docker image: `docker pull docker.elastic.co/elasticsearch/elasticsearch:8.18.3`
- Increase the max virtual memory: `sudo sysctl -w vm.max_map_count=262144`
- Start the Elasticsearch container: `docker run --name es01 --net elastic -p 9200:9200 -it -m 4GB docker.elastic.co/elasticsearch/elasticsearch:8.18.3`
- Note the generated password for the elastic user and enrollment token for Kibana displayed in the termina

In [3]:
load_dotenv()

True

In [4]:
from elasticsearch import Elasticsearch

es_password = os.environ.get('ES_PASSWORD')
es_client = Elasticsearch('http://localhost:9200', basic_auth=('elastic', es_password), verify_certs=False)                            

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))))

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [5]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

In [6]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [7]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = elastic_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

In [8]:
example = [
    [True, False, False, False, False], # 1, 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1 
    [False, False, True, False, False],  # 1/3
    [False, False, False, False, False], # 0
]

# 1 => 1
# 2 => 1 / 2 = 0.5
# 3 => 1 / 3 = 0.3333
# 4 => 0.25
# 5 => 0.2
# rank => 1 / rank
# none => 0

In [9]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [10]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [ ]:
hit_rate(example)

In [ ]:
mrr(example)

- Hit-Rate (recall)
- Mean Reciprocal Rank (mrr)

In [ ]:
hit_rate(relevance_total), mrr(relevance_total)

In [11]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [12]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [13]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/4622 [00:00<?, ?it/s]

In [14]:
hit_rate(relevance_total), mrr(relevance_total)

(0.771960190393769, 0.6615029568729269)

Compare with ES results:
```
(0.7395720769397017, 0.6032418413658963)
```

In [15]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [ ]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))

In [16]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

  0%|          | 0/4622 [00:00<?, ?it/s]

{'hit_rate': 0.771960190393769, 'mrr': 0.6615029568729269}